In [50]:
import yfinance as yf
import numpy as np
import torch

from torch import nn

In [3]:
from datetime import datetime

In [34]:
START = datetime(1990,1,1)
END = datetime(2020,6,5)

UVXY = yf.Ticker('UVXY')
hist = UVXY.history(start=START, end=END)
hist.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)

In [35]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [36]:
hist['RET'] = hist['Close'].pct_change()
hist.dropna(inplace=True)
hist['y'] = hist['RET'].rolling(10).sum()
hist['y_binary'] = (hist['y'] > 0).astype(int)
hist['y'] = hist['y'].shift(-9)
hist['y_binary'] = hist['y_binary'].shift(-9)
hist.dropna(inplace=True)
hist

,Open,High,Low,Close,RET,y,y_binary
Date,,,,,,,
2011-10-05,1.974600e+08,1.989000e+08,1.813800e+08,1.813800e+08,-0.118659,-0.405426,0.0
2011-10-06,1.811400e+08,1.878000e+08,1.740000e+08,1.740000e+08,-0.040688,-0.175707,0.0
2011-10-07,1.705800e+08,1.857600e+08,1.684800e+08,1.765200e+08,0.014483,-0.133360,0.0
2011-10-10,1.654800e+08,1.656000e+08,1.559400e+08,1.559400e+08,-0.116587,-0.249292,0.0
2011-10-11,1.570200e+08,1.572600e+08,1.491000e+08,1.506000e+08,-0.034244,-0.254364,0.0
...,...,...,...,...,...,...,...
2020-05-15,4.358000e+01,4.461000e+01,3.919000e+01,3.923000e+01,-0.037301,-0.176722,0.0
2020-05-18,3.529000e+01,3.643000e+01,3.466000e+01,3.544000e+01,-0.096610,-0.131081,0.0
2020-05-19,3.565000e+01,3.817000e+01,3.449000e+01,3.800000e+01,0.072235,-0.074649,0.0


In [45]:
in_seq1 = np.array(hist['Open'].values.reshape(-1,1))
in_seq2 = np.array(hist['High'].values.reshape(-1,1))
in_seq3 = np.array(hist['Low'].values.reshape(-1,1))
in_seq4 = np.array(hist['Close'].values.reshape(-1,1))


In [46]:
out_seq = np.array(hist['y'].values.reshape(-1,1))
out_seq_bin = np.array(hist['y_binary'].values.reshape(-1,1))

In [49]:
dataset = np.hstack((in_seq1, in_seq2, in_seq3, in_seq4, out_seq))
dataset_binary = np.hstack((in_seq1, in_seq2, in_seq3, in_seq4, out_seq_bin))

In [51]:
class MV_LSTM(nn.Module):
    def __init__(self, n_features, seq_length, hidden_dim=20, num_layers=1):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_length = seq_length
        self.n_hidden = hidden_dim
        self.n_layers = num_layers

        self.lstm = nn.LSTM(input_size=self.n_features,
                            hidden_size=self.n_hidden,
                            num_layers=self.n_layers,
                            batch_first=True)
        self.linear = nn.Linear(self.n_hidden*self.seq_length, 1)

    def init_hidden(self, batch_size):
        hidden_state = torch.zeros(self.n_layers, batch_size, self.n_hidden)
        cell_state = torch.zeros(self.n_layers, batch_size, self.n_hidden)
        self.hidden(hidden_state, cell_state)

    def forward(self, x):
        batch_size, seq_length, _ = x.size()

        lstm_out, self.hidden = self.lstm(x, self.hidden)
        x = lstm_out.contiguous().view(batch_size, -1)
        return self.linear(x)
        

In [52]:
MV_LSTM(4, 10, 20, 1)

MV_LSTM(
  (lstm): LSTM(4, 20, batch_first=True)
  (linear): Linear(in_features=200, out_features=1, bias=True)
)

In [53]:
n_features = 4
n_timesteps = 30

X, y = split_sequence(dataset, n_timesteps)
print(X.shape, y.shape)

(2142, 30, 4)(2142,)
